In [1]:
import numpy as np
import copy
import math
np.set_printoptions(precision=2)

<a name="toc_15456_3"></a>
## Model Prediction
$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
Model Prediction in vector notation (where $\cdot$ is a vector `dot product`):
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 

<a name="toc_15456_4"></a>
## Cost function
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 

- $f_{w,b}(x^{(i)})$ is the model's prediction for example $i$ using parameters $w,b$, while $y^{(i)}$ is the target value
- $(f_{w,b}(x^{(i)}) -y^{(i)})^2$ is the squared difference between the target value and the prediction.
- These differences are summed over all the $m$ examples and divided by `2m` to produce the cost, $J(w,b)$.  ($n$ is the number of features)
- We use squared error to penalize the model for bigger differences, we are emphasizing bigger errors this way. Also in other words, a lot of small errors is better than having one big error! To build a cost function that doesn't automatically get bigger as the training set size gets larger, by convention, we will compute the average squared error instead of the total squared error by dividing by m. By convention, the cost function that machine learning people use actually divides by 2 times m. The extra division by 2 is just meant to make some of our later calculations look neater, but the cost function still works whether you include this division by 2 or not.

<a name="toc_15456_5"></a>
## Gradient Descent

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

where, parameters $w_j$,  $b$, are updated simultaneously and where  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$



In [2]:
def predict(x, w, b): 
    """
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """

    p = np.dot(x, w) + b     
    return p    

In [3]:
def compute_cost(X, y, w, b): 
    """
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """

    m = X.shape[0]
    cost = 0.0

    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b
        cost += (f_wb_i - y[i]) ** 2
    cost /= 2 * m

    return cost

In [4]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """

    m, n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] += err * X[i, j]   # (6)
        dj_db += err                    # (7)
    dj_dw /= m                                
    dj_db /= m                                
    
    # this is more optimized
    # f_wb = X @ w + b              
    # e   = f_wb - y                
    # dj_dw  = (1/m) * (X.T @ e)
    # dj_db  = (1/m) * np.sum(e)

    return dj_db, dj_dw

In [5]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
    """

    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  # avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w, b)

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
      
        # Save cost J at each iteration
        if i < 100000:      # prevent resource exhaustion 
            J_history.append(cost_function(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i % math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}")
        
    return w, b, J_history #return final w,b and J history for graphing

In [1]:
# init training set and parameters
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])

# make a prediction
x1 = X_train[0,:]
y1 = y_train[0]
f_wb = predict(x1, w_init, b_init)
print(f"x1 value: {x1}")
print(f"prediction: {f_wb}, target value: {y1}\n")

# compute cost
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'Cost at optimal w : {cost}\n')

# compute gradient
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b:\n {tmp_dj_dw}\n')

# run gradient descent (all together)
w_init2 = np.zeros_like(w_init)
b_init2 = 0.
iterations = 1000
alpha = 5.0e-7
w_final, b_final, J_hist = gradient_descent(X_train, y_train, w_init2, b_init2,
                                                    compute_cost, compute_gradient, 
                                                    alpha, iterations)
print(f"\nb, w found by gradient descent: {b_final:0.2f},{w_final}")
m = X_train.shape[0]
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

NameError: name 'np' is not defined

*These results are not inspiring*! Cost is still declining and our predictions are not very accurate. The next lab will explore how to improve on this.

# Choosing Feature scaling and Learning Rate
The lectures discussed three different techniques: 
- Scaling to the range of -1 and 1 two ways: $x_i := \dfrac{x_i}{max} $ (for positive features), $x_i := \dfrac{x_i - min}{max - min} $ (for any features)
- Mean normalization: $x_i := \dfrac{x_i - \mu_i}{max - min} $ 
- Z-score normalization: $x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} $ (all features will have a mean of 0 and a standard deviation of 1.)

where $j$ selects a feature or a column in the $\mathbf{X}$ matrix. $µ_j$ is the mean of all the values for feature (j) and $\sigma_j$ is the standard deviation of feature (j).
$$
\begin{align}
\mu_j &= \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j \tag{5}\\
\sigma^2_j &= \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2  \tag{6}
\end{align}
$$

>**Implementation Note:** When normalizing the features, it is important
to store the values used for normalization - the mean value and the standard deviation used for the computations. After learning the parameters from the model, we often want to predict the prices of houses we have not seen before. Given a new x value (living room area and number of bed-rooms), we must first normalize x using the mean and standard deviation that we had previously computed from the training set.

**Implementation**

In [2]:
def zscore_normalize_features(X):
    """
    Args:
      X (ndarray (m, n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m, n)): input normalized by column
      mu (ndarray (n, ))     : mean of each feature
      sigma (ndarray (n, ))  : standard deviation of each feature
    """

    # find the mean of each column/feature
    mu     = np.mean(X, axis=0)
    # find the standard deviation of each column/feature
    sigma  = np.std(X, axis=0)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

In [3]:
def load_house_data():
    data = np.loadtxt("./data/houses.txt", delimiter=',', skiprows=1)
    X = data[:,:4]
    y = data[:,4]
    return X, y

X_train, y_train = load_house_data()
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Peak to Peak range by column in Raw        X: {np.ptp(X_train, axis=0)}")   
print(f"Peak to Peak range by column in Normalized X: {np.ptp(X_norm, axis=0)}")

NameError: name 'np' is not defined

In [ ]:
# a bit more iterations would even improve the cost even further
w_norm, b_norm, hist = gradient_descent(X_norm, y_train, w_init2, b_init2,
                                                    compute_cost, compute_gradient, 
                                                    1.0e-1, 119)


# Scikit-learn

In [4]:
import sklearn

X_train, y_train = load_house_data()
scaler = sklearn.preprocessing.StandardScaler()  # same as z-score
X_norm = scaler.fit_transform(X_train)

sgdr = sklearn.linear_model.SGDRegressor(max_iter=1000)
sgdr.fit(X_norm, y_train)
y_pred_sgd = sgdr.predict(X_norm)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
b_norm2 = sgdr.intercept_
w_norm2 = sgdr.coef_
print(f"Our model parameters:           w: {w_norm}, b: {b_norm}")
print(f"Scikit-learn model parameters:  w: {w_norm2}, b:{b_norm2}")

# predict target using normalized features
m = X_norm.shape[0]
yp = np.zeros(m)
for i in range(m):
    yp[i] = np.dot(X_norm[i], w_norm) + b_norm
print(f"Our prediction on training set:             {yp[:4]}")

y_pred = np.dot(X_norm, w_norm2) + b_norm2  
print(f"Scikit-learn prediction on training set:    {y_pred[:4]}")
print(f"Target values                               {y_train[:4]}\n")

print(f"number of iterations completed: {sgdr.n_iter_}, number of weight updates: {sgdr.t_}")
print(f"prediction using np.dot() and sgdr.predict match: {(y_pred == y_pred_sgd).all()}")